In [ ]:
import os
import json
import requests
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [ ]:
def authenticate():
    
    url = "https://auth.emsicloud.com/connect/token"

    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    SCOPE = os.getenv("SCOPE")
    payload = f"""client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&grant_type=client_credentials&scope={SCOPE}"""
    print(payload)
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.request("POST", url, data=payload, headers=headers)

    return json.loads(response.text)["access_token"]

In [ ]:
def get_skills(acess_token):

    url = url = "https://emsiservices.com/skills/versions/latest/skills"
    headers = {'Authorization': f"""Bearer {access_token}"""}
    response = requests.request("GET", url, headers=headers)
    path = os.path.join(os.getenv("DATA"),"Lightcast/Skills",f"""skills_{str(datetime.today())[:10]}.json""")
    with open(path, 'w') as file:
        json.dump(response.text, file, indent=4)
    


In [ ]:
def get_jobs(acess_token):

    url = "https://emsiservices.com/titles/versions/latest/titles"
    headers = {'Authorization': f"""Bearer {access_token}"""}
    response = requests.request("GET", url, headers=headers)
    path = os.path.join(os.getenv("DATA"),"Lightcast/Jobs",f"""jobs_{str(datetime.today())[:10]}.json""")
    with open(path, 'w') as file:
        json.dump(response.text, file, indent=4)


In [ ]:
access_token = authenticate()
get_skills(access_token)
get_jobs(access_token)

In [ ]:
get_skills(access_token)
get_jobs(access_token)
